# Comment Lister-Importer-Exporter des pdf vers un bucket S3


> TODO: Le Lecteur est invité dans son app Django a gérer les cas de conflit de nommage,a stocker les pdf en local (mis en cache) afin d'eviter de tous le temps soliciter le serveur cloud.A garder des logs de toute ces activités de concert avec l'object User Django

Source Django :
- https://codezup.com/integrating-aws-s3-file-handling-django-applications/
- https://dev.to/ericlecodeur/django-stocker-et-lire-des-fichiers-sur-amazon-s3-en-8-etapes-3gnh

### 1 Les packages

In [1]:
from dotenv import load_dotenv
import os
import boto3

### 2 Les variables d'environnement

In [2]:
load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY=os.getenv('AWS_SECRET_ACCESS_KEY')
region=os.getenv('region')
bucket_name=os.getenv('bucket_name')


### 3 Creation du Client S3

In [3]:
s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=region
)

### 4 Lister les fichiers disponible

In [9]:
def dataList_dispo():
    response = s3.list_objects_v2(Bucket=bucket_name)
    data_list = [item["Key"] for item in response["Contents"]]
    return data_list

dataList_dispo()

['data/hello_Word.jpg', 'theorie_des_jeux.m1.aei__0.pdf']

### 5 Envoyer un fichier vers le bucket

>Le 3iem argument de s3.uploiad_file() sera le nom du fichier dans S3

In [ ]:
def data_send(chemin):
    
    s3.upload_file(chemin,bucket_name,chemin)
data_send('data/hello_Word.jpg')

### 6 Telecharger un fichier depuis le bucket

In [11]:
def download_file( object_name, destination):
    s3.download_file(bucket_name, object_name, destination)


download_file( "theorie_des_jeux.m1.aei__0.pdf", "document_local.pdf")


### 7 Visualisation du pdf

>Le bucket est privé il faudra générer une url signée temporaire.
>En fonction de la charge de travail et des bessoins on pourra changer cela

In [14]:
def get_signed_url(object_name, expiration=3600):
    url = s3.generate_presigned_url(
        'get_object',
        Params={'Bucket': bucket_name, 'Key': object_name},
        ExpiresIn=expiration
    )
    return url


private_url = get_signed_url( "data/hello_Word.jpg")


In [16]:
from IPython.display import Image
Image(url=private_url)